data : https://www.kaggle.com/datasets/oddrationale/mnist-in-csv

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier

In [3]:
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam, SGD, RMSprop

In [4]:
directory = 'D:\\Pritesh\\Imarticus Learning\\Content\\Deep Learning\\Deep Learning\\Data\\'

In [ ]:
df = pd.read_csv(directory+'mnist_train.csv')

In [ ]:
df.head(2)

In [ ]:
df.shape

In [ ]:
# seperating label from the dataset, normalizing and reshaping the images to 28*28
# training dataset
train_labels = df['label'].values.reshape((-1, 1))
train_images = df.drop(columns=['label']).values

#### View as images

In [ ]:
train_images = train_images.reshape((-1, 28, 28)).astype('float32')/255

The images are reshaped into shape (28, 28) (shape for training data with 60000 examples = (60000, 28, 28)). '-1' automatically detects the overall size. It would be useful for plotting some images.

In [ ]:
train_images.shape

In [ ]:
def plot_images(images, labels, image_number):
    plt.imshow(images[image_number], cmap='gray')
    plt.title(f'label: {labels[image_number]}')

In [ ]:
plot_images(train_images, train_labels, 50000)

In [ ]:
### Reshape again to build model

In [ ]:
train_images = train_images.reshape((-1, 28*28))

#### Test data

In [ ]:
df_test = pd.read_csv(r'D:\Pritesh\Imarticus Learning\Content\Deep Learning\mnist_test.csv')

In [ ]:

test_labels = df_test['label'].values.reshape((-1, 1))
test_images = df_test.drop(columns=['label']).values

In [ ]:
# One-hot encode the labels
y_train = to_categorical(train_labels, 10)
y_test = to_categorical(test_labels, 10)


In [ ]:
y_train.shape

### Building a Neural network:

In [ ]:
normalizer = StandardScaler()
X_train_norm = normalizer.fit_transform(train_images)
X_test_norm = normalizer.fit_transform(test_images)

#### Attempt 1

In [ ]:
model_1 = Sequential()

In [ ]:
model_1.add(Dense(input_shape=(784,), units = 128, activation='relu'))

In [ ]:
model_1.add(Dense(units = 64, activation='relu'))

In [ ]:
model_1.add(Dropout(0.5))

In [ ]:
model_1.add(Dense(10, activation="softmax"))

In [ ]:
model_1.summary()

In [ ]:
model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
y_train.shape

In [ ]:
run_hist_1 = model_1.fit(X_train_norm, y_train, validation_data=(X_test_norm, y_test), epochs=10)

In [ ]:
#y_pred_class_nn_1 = model_1.predict(X_test_norm)
y_pred_prob_nn_1 = model_1.predict(X_test_norm)

In [ ]:
fig, ax = plt.subplots()
ax.plot(run_hist_1.history["loss"],'r', marker='.', label="Train Loss")
ax.plot(run_hist_1.history["val_loss"],'b', marker='.', label="Validation Loss")
ax.legend()

#### Attempt 2

In [ ]:
model = Sequential([
    Dense(128, input_shape = (784,), activation='relu'),  # First hidden layer with 128 neurons and ReLU activation.
    Dropout(0.5),  # Dropout layer after the first hidden layer
    Dense(10, activation='softmax')  # Output layer with 10 neurons (for 10 classes) and softmax activation.
])


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',  # Optimizer
              loss='categorical_crossentropy',  # Loss function for multi-class classification
              metrics=['accuracy'])  # List of metrics to be evaluated by the model during training and testing


In [ ]:
run_hist_2 =model.fit(X_train_norm, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
test_loss, test_acc = model.evaluate(X_test_norm, y_test)
print(f'Test accuracy: {test_acc}')

In [ ]:
fig, ax = plt.subplots()
ax.plot(run_hist_2.history["loss"],'r', marker='.', label="Train Loss")
ax.plot(run_hist_2.history["val_loss"],'b', marker='.', label="Validation Loss")
ax.legend()

In [ ]:
# from keras.callbacks import EarlyStopping

# early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# model.fit(X_train_norm, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


In [ ]:
# from keras.regularizers import l1_l2

# model = Sequential([
#     Flatten(input_shape=(28, 28)),
#     Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
#     Dense(10, activation='softmax')
# ])
